In [1]:
using PauliPropagation

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Warning: Replacing docs for `PauliPropagation._getprettystr :: Tuple{Dict, Int64}` in module `PauliPropagation`
└ @ Base.Docs docs/Docs.jl:243


## First pauli propagation and truncation test

In [2]:
nq = 64

64

In [9]:
pstr = PauliString(nq, :Z, round(Int, nq/2))

PauliString(nqubits: 64, 1.0 * IIIIIIIIIIIIIIIIIIII...)

In [10]:
nl = 4
topology = bricklayertopology(nq; periodic=false)
circuit = hardwareefficientcircuit(nq, nl; topology=topology)
nparams = countparameters(circuit)

1020

In [13]:
for i in 1:length(topology)
    print(topology[i])
end

(1, 2)(3, 4)(5, 6)(7, 8)(9, 10)(11, 12)(13, 14)(15, 16)(17, 18)(19, 20)(21, 22)(23, 24)(25, 26)(27, 28)(29, 30)(31, 32)(33, 34)(35, 36)(37, 38)(39, 40)(41, 42)(43, 44)(45, 46)(47, 48)(49, 50)(51, 52)(53, 54)(55, 56)(57, 58)(59, 60)(61, 62)(63, 64)(2, 3)(4, 5)(6, 7)(8, 9)(10, 11)(12, 13)(14, 15)(16, 17)(18, 19)(20, 21)(22, 23)(24, 25)(26, 27)(28, 29)(30, 31)(32, 33)(34, 35)(36, 37)(38, 39)(40, 41)(42, 43)(44, 45)(46, 47)(48, 49)(50, 51)(52, 53)(54, 55)(56, 57)(58, 59)(60, 61)(62, 63)

In [5]:
using Random
Random.seed!(42)
thetas = randn(nparams);


UndefVarError: UndefVarError: `nparams` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [18]:
@time psum = propagate(circuit, pstr, thetas)

  0.620217 seconds (2.17 k allocations: 33.391 MiB, 9.20% gc time)


PauliSum(nqubits: 64, 164692 Pauli terms:
 4.027e-9 * IIIIIIIIIIIIIIIIIIII...
 -5.3954e-9 * IIIIIIIIIIIIIIIIIIII...
 5.4961e-5 * IIIIIIIIIIIIIIIIIIII...
 6.1435e-5 * IIIIIIIIIIIIIIIIIIII...
 3.0334e-8 * IIIIIIIIIIIIIIIIIIII...
 5.3742e-9 * IIIIIIIIIIIIIIIIIIII...
 -4.3295e-8 * IIIIIIIIIIIIIIIIIIII...
 1.6012e-8 * IIIIIIIIIIIIIIIIIIII...
 -2.4033e-8 * IIIIIIIIIIIIIIIIIIII...
 7.8656e-7 * IIIIIIIIIIIIIIIIIIII...
 -9.4308e-5 * IIIIIIIIIIIIIIIIIIII...
 -2.3263e-6 * IIIIIIIIIIIIIIIIIIII...
 -1.4941e-8 * IIIIIIIIIIIIIIIIIIII...
 2.2601e-9 * IIIIIIIIIIIIIIIIIIII...
 -8.2387e-7 * IIIIIIIIIIIIIIIIIIII...
 -7.4329e-9 * IIIIIIIIIIIIIIIIIIII...
 -1.06e-7 * IIIIIIIIIIIIIIIIIIII...
 -4.3301e-8 * IIIIIIIIIIIIIIIIIIII...
 -5.4605e-7 * IIIIIIIIIIIIIIIIIIII...
 -4.3812e-6 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

In [19]:
overlapwithzero(psum)

0.31961629594501484

In [20]:
max_weight = 5
@time psum_weight = propagate(circuit, pstr, thetas; max_weight = max_weight)

  0.148307 seconds (123.65 k allocations: 8.190 MiB, 51.29% compilation time)


PauliSum(nqubits: 64, 16453 Pauli terms:
 -1.6207e-5 * IIIIIIIIIIIIIIIIIIII...
 0.011987 * IIIIIIIIIIIIIIIIIIII...
 7.3032e-5 * IIIIIIIIIIIIIIIIIIII...
 7.8792e-8 * IIIIIIIIIIIIIIIIIIII...
 6.5435e-10 * IIIIIIIIIIIIIIIIIIII...
 -9.1272e-9 * IIIIIIIIIIIIIIIIIIII...
 1.8143e-8 * IIIIIIIIIIIIIIIIIIII...
 1.7645e-10 * IIIIIIIIIIIIIIIIIIII...
 0.00019728 * IIIIIIIIIIIIIIIIIIII...
 0.00018569 * IIIIIIIIIIIIIIIIIIII...
 0.00034955 * IIIIIIIIIIIIIIIIIIII...
 7.9429e-6 * IIIIIIIIIIIIIIIIIIII...
 0.0016765 * IIIIIIIIIIIIIIIIIIII...
 -1.2376e-8 * IIIIIIIIIIIIIIIIIIII...
 2.6325e-7 * IIIIIIIIIIIIIIIIIIII...
 -0.00047972 * IIIIIIIIIIIIIIIIIIII...
 6.0018e-10 * IIIIIIIIIIIIIIIIIIII...
 0.00088951 * IIIIIIIIIIIIIIIIIIII...
 -4.7825e-6 * IIIIIIIIIIIIIIIIIIII...
 -9.0788e-5 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

## Custom gates

In [21]:
struct SWAPgate <: StaticGate
    qinds::Tuple{Int, Int}
end

In [22]:
# define apply function for SWAPgate
function PauliPropagation.apply(gate::SWAPgate, pstr, theta, coefficients; kwargs...)
    # get Pauli on the first site 
    pauli1 = getpauli(pstr, gate.qinds[1])
    # get Pauli on the second site 
    pauli2 = getpauli(pstr, gate.qinds[2])

    # set the Pauli on the first site to the second Pauli
    pstr = setpauli(pstr, pauli2, gate.qinds[1])
    # set the Pauli on the first site to the first Pauli 
    pstr = setpauli(pstr, pauli1, gate.qinds[2])

    return pstr, coefficients
end

In [29]:
nq = 21
# 1d linear topology. default periodic: false
topology = bricklayertopology(nq);

In [30]:
nl = 3 
base_circuit = tfitrottercircuit(nq, nl; topology=topology)
nparams = countparameters(base_circuit)

123

In [31]:
pstr = PauliString(nq, [:Z, :Z], [7, 13])

PauliString(nqubits: 21, 1.0 * IIIIIIZIIIIIZIIIIIII...)

In [32]:
using Random
Random.seed!(42)
thetas = randn(nparams)

123-element Vector{Float64}:
 -0.36335748145177754
  0.2517372155742292
 -0.31498797116895605
 -0.31125240132442067
  0.8163067649323273
  0.47673837983187795
 -0.8595553820616212
 -1.4692882055065464
 -2.114334831130985
  0.04378166203061088
 -0.8253345499750688
  0.840288523245819
  0.4338858743048611
  ⋮
 -0.27271841265920194
 -0.8351828409416799
  0.9552537029900053
  1.2438797492400726
  0.6091662702133595
 -0.3407882587838508
 -1.8623610290958061
 -1.8071566859269461
 -0.30215190432626304
 -0.20661311448119266
 -0.31074387308373413
 -0.040473400248354906

In [33]:
max_weight = 4

4

In [34]:
@time base_psum = propagate(base_circuit, pstr, thetas; max_weight = max_weight)
base_psum

  0.152108 seconds (368.86 k allocations: 18.251 MiB, 2.52% gc time, 99.72% compilation time)


PauliSum(nqubits: 21, 194 Pauli terms:
 4.5691e-6 * IIIIIIYIIIIZIYZIIIII...
 0.0039723 * IIIIIIXYIIIIZIIIIIII...
 1.6824e-6 * IIIIIYXIIIIXIYIIIIII...
 -1.7443e-5 * IIIIYXIXIIIIZIIIIIII...
 -0.060822 * IIIIIZXIIIIIXZIIIIII...
 0.0010823 * IIIIIIXYIIIYXIIIIIII...
 0.067264 * IIIIIIXZIIIYXIIIIIII...
 0.00020843 * IIIIIXYXIIIIZIIIIIII...
 0.0024923 * IIIIIYXIIIIIXYIIIIII...
 0.047715 * IIIIIIZIIIIZZZIIIIII...
 1.1128e-5 * IIIIXIIYIIIIYIIIIIII...
 0.034974 * IIIIIZZZIIIIZIIIIIII...
 0.002254 * IIIIIXZXIIIIZIIIIIII...
 0.0015483 * IIIIIIZIIIIZYYIIIIII...
 0.020889 * IIIIIIYIIIIXIYIIIIII...
 0.040219 * IIIIIIZIIIIYYZIIIIII...
 7.5591e-7 * IIIIIIYIIIIYIYZIIIII...
 -7.0324e-5 * IIIIXIIZIIIIYIIIIIII...
 0.00033187 * IIIIIIZIIIIZIXIIIIII...
 -2.2795e-5 * IIIIZZIZIIIIZIIIIIII...
  ⋮)

## Create clifford gates of Algo circuit

In [24]:
push!(clifford_map, :ECR => [(1, 0x00), (-1, 0x06), (-1, 0x05), (-1, 0x03), (-1, 0x02), (-1, 0x01), (1, 0x02), (-1, 0x07), (-1, 0x0f), (1, 0x09), (-1, 0x0a), (1, 0x0c), (1, 0x0b), (-1, 0x0d), (1, 0x0e), (1, 0x08)])

Dict{Symbol, Vector{Tuple{Int64, UInt8}}} with 10 entries:
  :Z        => [(1, 0x00), (-1, 0x01), (-1, 0x02), (1, 0x03)]
  :ZZpihalf => [(1, 0x00), (1, 0x0e), (-1, 0x0d), (1, 0x03), (1, 0x0b), (1, 0x0…
  :SX       => [(1, 0x00), (1, 0x01), (-1, 0x03), (1, 0x02)]
  :ECR      => [(1, 0x00), (-1, 0x06), (-1, 0x05), (-1, 0x03), (-1, 0x02), (-1,…
  :H        => [(1, 0x00), (1, 0x03), (-1, 0x02), (1, 0x01)]
  :CNOT     => [(1, 0x00), (1, 0x05), (1, 0x06), (1, 0x03), (1, 0x04), (1, 0x01…
  :S        => [(1, 0x00), (-1, 0x02), (1, 0x01), (1, 0x03)]
  :X        => [(1, 0x00), (1, 0x01), (-1, 0x02), (-1, 0x03)]
  :Y        => [(1, 0x00), (-1, 0x01), (1, 0x02), (1, 0x03)]
  :SWAP     => [(1, 0x00), (1, 0x04), (1, 0x08), (1, 0x0c), (1, 0x01), (1, 0x05…

In [19]:
sx_map = createcliffordmap(Dict(
    (:I,) => (1, :I),
    (:X,) => (1, :X),
    (:Y,) => (-1, :Z),
    (:Z,) => (1, :Y)
))

4-element Vector{Tuple{Int64, UInt8}}:
 (1, 0x00)
 (1, 0x01)
 (-1, 0x03)
 (1, 0x02)

In [21]:
push!(clifford_map, :SX => sx_map)

Dict{Symbol, Vector{Tuple{Int64, UInt8}}} with 9 entries:
  :Z        => [(1, 0x00), (-1, 0x01), (-1, 0x02), (1, 0x03)]
  :ZZpihalf => [(1, 0x00), (1, 0x0e), (-1, 0x0d), (1, 0x03), (1, 0x0b), (1, 0x0…
  :SX       => [(1, 0x00), (1, 0x01), (-1, 0x03), (1, 0x02)]
  :H        => [(1, 0x00), (1, 0x03), (-1, 0x02), (1, 0x01)]
  :CNOT     => [(1, 0x00), (1, 0x05), (1, 0x06), (1, 0x03), (1, 0x04), (1, 0x01…
  :S        => [(1, 0x00), (-1, 0x02), (1, 0x01), (1, 0x03)]
  :X        => [(1, 0x00), (1, 0x01), (-1, 0x02), (-1, 0x03)]
  :Y        => [(1, 0x00), (-1, 0x01), (1, 0x02), (1, 0x03)]
  :SWAP     => [(1, 0x00), (1, 0x04), (1, 0x08), (1, 0x0c), (1, 0x01), (1, 0x05…

In [26]:
clifford_map

Dict{Symbol, Vector{Tuple{Int64, UInt8}}} with 10 entries:
  :Z        => [(1, 0x00), (-1, 0x01), (-1, 0x02), (1, 0x03)]
  :ZZpihalf => [(1, 0x00), (1, 0x0e), (-1, 0x0d), (1, 0x03), (1, 0x0b), (1, 0x0…
  :SX       => [(1, 0x00), (1, 0x01), (-1, 0x03), (1, 0x02)]
  :ECR      => [(1, 0x00), (-1, 0x06), (-1, 0x05), (-1, 0x03), (-1, 0x02), (-1,…
  :H        => [(1, 0x00), (1, 0x03), (-1, 0x02), (1, 0x01)]
  :CNOT     => [(1, 0x00), (1, 0x05), (1, 0x06), (1, 0x03), (1, 0x04), (1, 0x01…
  :S        => [(1, 0x00), (-1, 0x02), (1, 0x01), (1, 0x03)]
  :X        => [(1, 0x00), (1, 0x01), (-1, 0x02), (-1, 0x03)]
  :Y        => [(1, 0x00), (-1, 0x01), (1, 0x02), (1, 0x03)]
  :SWAP     => [(1, 0x00), (1, 0x04), (1, 0x08), (1, 0x0c), (1, 0x01), (1, 0x05…

## Testing importing circuit from qiskit

In [27]:
using CSV, DataFrames

In [30]:
qcdf = CSV.read("ki_circ_11.csv", DataFrame; delim = ",", header = false)

Row,Column1,Column2,Column3
,String3,String31,String7
1,rz,[-1.5707963267948968],[0]
2,sx,[],[0]
3,rz,[4.0123889803846895],[0]
4,sx,[],[0]
5,rz,[10.995574287564276],[0]
6,rz,[-3.141592653589793],[1]
7,sx,[],[1]
8,rz,[4.71238898038469],[1]
9,sx,[],[1]


In [44]:
qcdf.Column2[1]

"[-1.5707963267948968]"

In [50]:
for i in length(qcdf.Column2)
    qcdf.Column2[i] = parse(Float64,replace(qcdf.Column2[i], "[" => "", "]" => ""))
end

LoadError: MethodError: [0mCannot `convert` an object of type [92mFloat64[39m[0m to an object of type [91mString31[39m
The function `convert` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:AbstractString
[0m[90m   @[39m [90mBase[39m [90mstrings/[39m[90m[4mbasic.jl:231[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:126[24m[39m
[0m  convert(::Type{T}, [91m::AbstractString[39m) where T<:AbstractString
[0m[90m   @[39m [90mBase[39m [90mstrings/[39m[90m[4mbasic.jl:232[24m[39m


In [51]:
parse(Float16, "")

LoadError: ArgumentError: cannot parse "" as Float16

## Creating algo circuit

In [7]:
function ki_circ_modified(L::Integer,J::Float64, b::Float64, num_steps::Integer, hi::Float64)
    circuit::Vector{Gate} = []
    thetas::Vector{Float64} = []

    push!(circuit, CliffordGate(:H, 1))

    for q in 2:2:L
        push!(circuit, CliffordGate(:H, q))
        push!(circuit, CliffordGate(:X, q+1))
        push!(circuit, CliffordGate(:CNOT, (q,q+1)))
    end

    for jj in 1:num_steps
        print("Step :",jj,"\n")
        print("jj-1%2 = ", jj-1%2, "\n")
        if (jj-1)% 2 == 0
            print("Biglayer\n")
            for ii in 1:2:L-1
                # RZZ
                push!(circuit, PauliGate([:Z,:Z], (ii, ii+1)))
                push!(thetas,  2*J)

                #RZ
                push!(circuit, PauliGate(:Z, ii))
                push!(thetas, hi)

                #RX
                push!(circuit, PauliGate(:X, ii))
                push!(thetas, 2*b)
                push!(circuit, PauliGate(:X, ii+1))
                push!(thetas, 2*b)

                # RZZ
                push!(circuit, PauliGate([:Z,:Z], (ii, ii+1)))
                push!(thetas,  2*J)

                #RZ
                push!(circuit, PauliGate(:Z, ii))
                push!(thetas, hi)
            end
        

        elseif (jj-1)%2 == 1
            print("Smalllayer\n")
            for ii in 1:2:L-2
                # RZZ
                push!(circuit, PauliGate([:Z,:Z], (ii+1, ii+2)))
                push!(thetas,  2*J)
                

                #RZ
                push!(circuit, PauliGate(:Z, ii+1))
                push!(thetas, hi)

                #RX
                push!(circuit, PauliGate(:X, ii+1))
                push!(thetas, 2*b)
                push!(circuit, PauliGate(:X, ii+2))
                push!(thetas, 2*b)

                # RZZ
                push!(circuit, PauliGate([:Z,:Z], (ii+1, ii+2)))
                push!(thetas,  2*J)

                #RZ
                push!(circuit, PauliGate(:Z, ii+1))
                push!(thetas, hi)
            end
        end
    end

    return circuit, thetas

end



ki_circ_modified (generic function with 1 method)

## Propagate in Algo circuit

In [16]:
nq = 21
pstr = PauliString(nq, :X, round(Int, nq/2))

PauliString(nqubits: 21, 1.0 * IIIIIIIIIXIIIIIIIIII...)

In [17]:
circuit, thetas = ki_circ_modified(21, pi/4, pi/4, 10, 0.5);
countparameters(circuit)

600

In [28]:
psum = propagate(circuit, pstr, thetas; max_weight = 2 )

PauliSum(nqubits: 21, 1 Pauli term: 
 0.27094 * IIIIIIIIIIIIIIIIIIIZ...
)

In [29]:
overlapwithzero(psum)

0.0

In [39]:
_getprettystr(psum.op_dict, 21)

"1 Pauli term: \n 0.27094 * IIIIIIIIIIIIIIIIIIIZX\n"

In [38]:
function _getprettystr(psum::Dict, nqubits::Int; max_lines=40)
    str = ""
    header = length(psum) == 1 ? "1 Pauli term: \n" : "$(length(psum)) Pauli terms:\n"
    str *= header

    for (ii, (op, coeff)) in enumerate(psum)
        if ii > max_lines
            new_str = "  ⋮"
            str *= new_str
            break
        end
        pauli_string = inttostring(op, nqubits)
        if length(pauli_string) > 40
            pauli_string = pauli_string[1:20] * "..."
        end
        if isa(coeff, Number)
            coeff_str = round(coeff, sigdigits=5)
        elseif isa(coeff, PathProperties)
            if isa(coeff.coeff, Number)
                coeff_str = "PathProperty($(round(coeff.coeff, sigdigits=5)))"
            else
                coeff_str = "PathProperty($(typeof(coeff.coeff)))"
            end
        else
            coeff_str = "($(typeof(coeff)))"
        end
        new_str = " $(coeff_str) * $(pauli_string)\n"
        str *= new_str
    end

    return str

end

_getprettystr (generic function with 1 method)